In [3]:
# --- 1. 환경 설정 셀 (수정된 버전) ---
import os
import sys

# 작업 디렉토리를 올바른 위치로 변경
workspace_path = '/workspace'
os.chdir(workspace_path)

# RFdiffusion 경로를 파이썬이 인식하도록 추가
rfdiffusion_path = os.path.join(workspace_path, 'RFdiffusion')
if rfdiffusion_path not in sys.path:
    sys.path.append(rfdiffusion_path)
import time
import signal
import sys
import random
import string
import re
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

# Colab이 아닌 로컬 환경이므로, RFdiffusion 경로를 직접 추가
if 'RFdiffusion' not in sys.path:
    home_dir = os.path.expanduser("~")
    rfdiffusion_path = os.path.join(home_dir, 'RFdiffusion')
    sys.path.append(rfdiffusion_path)

os.environ["DGLBACKEND"] = "pytorch"

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

# --- Colab의 files.upload()를 대체하는 로컬 파일 처리 함수 ---
def get_pdb(pdb_code=None, use_upload=False):
    if use_upload:
        upload_widget = widgets.FileUpload(
            accept='.pdb', description='PDB 파일 업로드', button_style='info'
        )
        display(upload_widget)
        
        def wait_for_upload(widget):
            while len(widget.value) == 0: time.sleep(0.1)
            uploaded_filename = list(widget.value.keys())[0]
            pdb_content = widget.value[uploaded_filename]['content']
            pdb_filename = "tmp.pdb"
            with open(pdb_filename, "wb") as out: out.write(pdb_content)
            print(f"'{uploaded_filename}'이(가) 'tmp.pdb'로 저장되었습니다.")
            return pdb_filename
            
        return wait_for_upload(upload_widget)
    elif pdb_code is None or pdb_code == "":
        print("PDB 코드를 입력하거나 use_upload=True로 설정하세요.")
        return None
    elif os.path.isfile(pdb_code): return pdb_code
    elif len(pdb_code) == 4:
        pdb_filename = f"{pdb_code}.pdb1"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz && gunzip -f {pdb_code}.pdb1.gz")
        return pdb_filename
    else:
        pdb_filename = f"AF-{pdb_code}-F1-model_v4.pdb"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/{pdb_filename}")
        return pdb_filename

def run_ananas(pdb_str, path, sym=None):
    pdb_filename = f"outputs/{path}/ananas_input.pdb"
    out_filename = f"outputs/{path}/ananas.json"
    os.makedirs(f"outputs/{path}", exist_ok=True)
    with open(pdb_filename,"w") as handle: handle.write(pdb_str)
    cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
    if sym is None: os.system(cmd)
    else: os.system(f"{cmd} {sym}")
    try:
        with open(out_filename,"r") as f: out = json.load(f)
        results, AU = out[0], out[-1]["AU"]
        group, chains, rmsd = AU["group"], AU["chain names"], results["Average_RMSD"]
        print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")
        C = np.array(results['transforms'][0]['CENTER'])
        A = [np.array(t["AXIS"]) for t in results['transforms']]
        new_lines = []
        for line in pdb_str.split("\n"):
            if line.startswith("ATOM"):
                chain = line[21:22]
                if chain in chains:
                    x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
                    if group[0] == "c": x = sym_it(x,C,A[0])
                    if group[0] == "d": x = sym_it(x,C,A[1],A[0])
                    coord_str = "".join([f"{a:8.3f}" for a in x])
                    new_lines.append(line[:30]+coord_str+line[54:])
            else: new_lines.append(line)
        return results, "\n".join(new_lines)
    except Exception as e:
        print(f"AnAnaS 결과 처리 중 오류 발생: {e}")
        return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):
    def run_command_and_get_pid(command):
        pid_file = '/dev/shm/pid'
        os.system(f'nohup {command} > /dev/null & echo $! > {pid_file}')
        with open(pid_file, 'r') as f: pid = int(f.read().strip())
        os.remove(pid_file)
        return pid
    def is_process_running(pid):
        try: os.kill(pid, 0)
        except OSError: return False
        else: return True
    run_output = widgets.Output()
    progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
    display(widgets.VBox([progress, run_output]))
    for n in range(steps):
        if os.path.isfile(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
    pid = run_command_and_get_pid(command)
    try:
        fail = False
        for _ in range(num_designs):
            for n in range(steps):
                wait = True
                while wait and not fail:
                    time.sleep(0.1)
                    if os.path.isfile(f"/dev/shm/{n}.pdb"):
                        with open(f"/dev/shm/{n}.pdb", "r") as f: pdb_str = f.read()
                        if "TER" in pdb_str or "ENDMDL" in pdb_str: wait = False
                        elif not is_process_running(pid): fail = True
                    elif not is_process_running(pid): fail = True
                if fail:
                    progress.bar_style = 'danger'; progress.description = "failed"
                    break
                else:
                    progress.value = (n+1) / steps
                    if visual != "none":
                        with run_output:
                            run_output.clear_output(wait=True)
                            if visual == "image":
                                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                                fig = plt.figure(); fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1); plt.show()
                            if visual == "interactive":
                                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                                view.addModel(pdb_str,'pdb'); view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                                view.zoomTo(); view.show()
                if os.path.exists(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
            if fail:
                progress.bar_style = 'danger'; progress.description = "failed"
                break
        while is_process_running(pid): time.sleep(0.1)
    except KeyboardInterrupt:
        os.kill(pid, signal.SIGTERM); progress.bar_style = 'danger'; progress.description = "stopped"
    
def run_diffusion(contigs, path, pdb=None, iterations=50, symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, num_designs=1, visual="none"):
    full_path = f"outputs/{path}"; os.makedirs(full_path, exist_ok=True)
    opts = [f"inference.output_prefix={full_path}", f"inference.num_designs={num_designs}"]
        # --- 👇 여기 이 한 줄을 추가하세요! ---
    opts.append(f"+inference.schedule_directory_path={full_path}/schedules")
    # ---
    if chains == "": chains = None
    if symmetry in ["auto","cyclic","dihedral"]:
        if symmetry == "auto": sym, copies = None, 1
        else: sym, copies = {"cyclic":(f"c{order}",order), "dihedral":(f"d{order}",order*2)}[symmetry]
    else: symmetry = None; sym, copies = None, 1
    contigs_list = contigs.replace(","," ").replace(":"," ").split()
    is_fixed, is_free = False, False; fixed_chains = []
    for contig in contigs_list:
        for x in contig.split("/"):
            a = x.split("-")[0]
            if a and a[0].isalpha():
                is_fixed = True
                if a[0] not in fixed_chains: fixed_chains.append(a[0])
            if a.isnumeric(): is_free = True
    if len(contigs_list) == 0 or not is_free: mode = "partial"
    elif is_fixed: mode = "fixed"
    else: mode = "free"
    if mode in ["partial","fixed"]:
        pdb_str = pdb_to_string(pdb, chains=chains)
        if symmetry == "auto":
            a, pdb_str = run_ananas(pdb_str, path)
            if a is None: print(f'ERROR: no symmetry detected'); symmetry = None; sym, copies = None, 1
            else:
                if a["group"][0] == "c": symmetry = "cyclic"; sym, copies = a["group"], int(a["group"][1:])
                elif a["group"][0] == "d": symmetry = "dihedral"; sym, copies = a["group"], 2 * int(a["group"][1:])
                else: print(f'ERROR: detected symm ({a["group"]}) not supported'); symmetry = None; sym, copies = None, 1
        elif mode == "fixed": pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)
        pdb_filename = f"{full_path}/input.pdb"
        with open(pdb_filename, "w") as handle: handle.write(pdb_str)
        parsed_pdb = parse_pdb(pdb_filename)
        opts.append(f"inference.input_pdb={pdb_filename}")
        if mode in ["partial"]:
            iterations = int(80 * (iterations / 200)); opts.append(f"diffuser.partial_T={iterations}")
            contigs_list = fix_partial_contigs(contigs_list, parsed_pdb)
        else: opts.append(f"diffuser.T={iterations}"); contigs_list = fix_contigs(contigs_list, parsed_pdb)
    else:
        opts.append(f"diffuser.T={iterations}"); parsed_pdb = None
        contigs_list = fix_contigs(contigs_list, parsed_pdb)
    if hotspot is not None and hotspot != "": opts.append(f"ppi.hotspot_res=[{hotspot}]")
    if sym is not None:
        sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
        if add_potential: sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'", "potentials.olig_intra_all=True","potentials.olig_inter_all=True", "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
        opts = sym_opts + opts; contigs_list = sum([contigs_list] * copies,[])
    opts.append(f"'contigmap.contigs=[{' '.join(contigs_list)}]'")
    opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]
    print("mode:", mode); print("output:", full_path); print("contigs:", contigs_list)
    run_script_path = "/workspace/RFdiffusion/run_inference.py"
    opts_str = " ".join(opts)
    cmd = f"python {run_script_path} {opts_str}"
    print(cmd)
    run(cmd, iterations, num_designs, visual=visual)
    for n in range(num_designs):
        pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb", f"outputs/traj/{path}_{n}_Xt-1_traj.pdb", f"{full_path}_{n}.pdb"]
        for pdb_file in pdbs:
            if os.path.exists(pdb_file):
                with open(pdb_file,"r") as handle: pdb_str = handle.read()
                with open(pdb_file,"w") as handle: handle.write(fix_pdb(pdb_str, contigs_list))
    return contigs_list, copies

print("✅ Setup cell is ready.")

✅ Setup cell is ready.


In [4]:
# --- 바인더 디자인 실행 셀 (수정된 버전) ---

# 1. 파라미터 설정
name = "0902-50-test"
contigs = "C311-391/0 100-100"
hotspot = "C331,C360,C378,C381,C323"
iterations = 50
num_designs = 5
visual = "image"

# --- 2. 업로드한 PDB 파일 경로 직접 지정 ---
# 여기에 1단계에서 업로드한 PDB 파일의 정확한 이름을 입력하세요.
pdb_filename = "4k9e_c.pdb" 
pdb = os.path.join("/workspace", pdb_filename)

# 3. RFdiffusion 실행
# --- 이 아랫부분이 수정되었습니다 ---

# 기본 실행 이름(폴더명으로 사용될)을 설정합니다.
base_name = name
output_dir = f"outputs/{base_name}"

# 만약 동일한 이름의 폴더가 이미 존재하면, 중복을 피하기 위해 랜덤 문자열을 뒤에 붙입니다.
while os.path.exists(output_dir):
    random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
    base_name = f"{name}_{random_suffix}"
    output_dir = f"outputs/{base_name}"

# run_diffusion 함수는 이 path를 파일 이름의 접두사로 사용합니다.
# 예: path가 "8051/8051" 이면, 결과는 outputs/8051/8051_0.pdb 로 저장됩니다.
path = os.path.join(base_name, base_name)

print(f"✅ 결과가 저장될 폴더: {output_dir}")
# --- 수정된 부분 끝 ---


flags = {
    "contigs": contigs, "pdb": pdb, "iterations": int(iterations),
    "hotspot": hotspot, "path": path, "num_designs": int(num_designs),
    "visual": visual, "symmetry": "none", "order": 1, "chains": "", "add_potential": True
}

for k, v in flags.items():
    if isinstance(v, str):
        flags[k] = v.replace("'", "").replace('"', '')

contigs, copies = run_diffusion(**flags)

# --- 마지막 출력 메시지도 수정되었습니다 ---
print(f"✅ RFdiffusion 실행 완료! 결과는 '{output_dir}' 폴더에 저장되었습니다.")

✅ 결과가 저장될 폴더: outputs/0902-50-test
mode: fixed
output: outputs/0902-50-test/0902-50-test
contigs: ['C311-391', '100-100']
python /workspace/RFdiffusion/run_inference.py inference.output_prefix=outputs/0902-50-test/0902-50-test inference.num_designs=5 +inference.schedule_directory_path=outputs/0902-50-test/0902-50-test/schedules inference.input_pdb=outputs/0902-50-test/0902-50-test/input.pdb diffuser.T=50 ppi.hotspot_res=[C331,C360,C378,C381,C323] 'contigmap.contigs=[C311-391 100-100]' inference.dump_pdb=True inference.dump_pdb_path='/dev/shm'


/workspace/RFdiffusion/util_module.py:259: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/Cross.cpp:62.)
  CBrotaxis1 = (CBr-CAr).cross(NCr-CAr)


✅ RFdiffusion 실행 완료! 결과는 'outputs/0902-50-test' 폴더에 저장되었습니다.


In [ ]:
%%time
import os
import sys
import pandas as pd
import glob

# @markdown ### **👇 여기에 이전 실행 정보를 입력하세요.**
# @markdown ---

# <<< (1) RFdiffusion 실행 시 사용했던 `name` 변수 값을 입력하세요.
# 결과가 저장된 폴더의 이름입니다. (예: "0811-200-200")
run_name = "0829-150-600sc"  # @param {type:"string"}

# <<< (2) 위 백본들을 생성할 때 사용했던 '정확한' Contig를 입력하세요.
# 이 정보는 코드가 스스로 알 수 없으므로, 반드시 올바르게 입력해야 합니다.
contigs_for_run = "C311-391/0 100-100"  # @param {type:"string"}

# @markdown ---
# @markdown ### **MPNN 및 AlphaFold 설정**
num_seqs = 8      #@param {type:"raw"}
num_recycles = 3  #@param {type:"raw"}
initial_guess = True
use_multimer = True
rm_aa = "C"
mpnn_sampling_temp = 0.1

# --- 아래는 수정할 필요 없이 그대로 실행하면 됩니다. ---

# (A) 이전 실행 정보를 바탕으로 변수들을 다시 만듭니다.
path = os.path.join(run_name, run_name)
contigs_str = ":".join(contigs_for_run.replace("'", "").replace('"', '').split())
copies = 1

# (B) PDB 파일 개수를 자동으로 셉니다.
pdb_path_pattern = f"outputs/{path}_*.pdb"
pdb_files = glob.glob(pdb_path_pattern)
num_designs = len(pdb_files)

if num_designs == 0:
    print(f"🚨 에러: '{os.path.dirname(pdb_path_pattern)}' 폴더에서 PDB 파일을 찾을 수 없습니다. 'run_name'을 확인해주세요.")
else:
    print(f"✅ 총 {num_designs}개의 PDB 파일을 찾았습니다. 검증 작업을 시작합니다.")
    
    # (C) designability_test 스크립트 실행
    opts = [f"--pdb=outputs/{path}_0.pdb",
            f"--loc=outputs/{path}",
            f"--contig={contigs_str}",
            f"--copies={copies}",
            f"--num_seqs={num_seqs}",
            f"--num_recycles={num_recycles}",
            f"--rm_aa={rm_aa}",
            f"--mpnn_sampling_temp={mpnn_sampling_temp}",
            f"--num_designs={num_designs}"]
    if initial_guess: opts.append("--initial_guess")
    if use_multimer: opts.append("--use_multimer")
    opts_str = ' '.join(opts)
    
    get_ipython().system(f"python -m colabdesign.rf.designability_test {opts_str}")

    # (D) 결과 요약 출력
    best_pdb_path = f"outputs/{path}/best.pdb"
    all_results_path = f"outputs/{path}/all_results.csv"

    print("\n----------------------------------------------------")
    try:
        if os.path.exists(all_results_path):
            print("📊 전체 디자인 결과 요약:")
            df = pd.read_csv(all_results_path)
            print(df[['design', 'n', 'plddt', 'rmsd']].to_string(index=False))
            print("----------------------------------------------------")

        with open(best_pdb_path, "r") as f:
            info = f.readline().strip().split()
            design_num = info[3]
            seq_num = info[5]
            rmsd_val = info[7]

        print(f"🏆 Best 모델이 선정되었습니다! 🏆")
        print(f"   - 모델 번호: design {design_num} / sequence {seq_num}")
        print(f"   - RMSD 값: {rmsd_val}")
        print("----------------------------------------------------")

    except FileNotFoundError:
        print("⚠️ 'best.pdb' 또는 'all_results.csv' 파일을 찾을 수 없습니다. 스크립트 실행에 문제가 있었을 수 있습니다.")

📂 모든 결과는 'outputs/0903-binder-final' 폴더에 저장됩니다.
⏳ RFdiffusion 프로세스를 시작합니다. 시간이 오래 걸릴 수 있습니다.
   (자세한 로그는 '/workspace/rfdiffusion_log.txt' 파일에 저장됩니다)


In [2]:
# --- 최종 통합 스크립트 v5 (MPNN Contig 형식 최종 수정) ---

# --- 1. 환경 설정 ---
import os
import sys
import time
import pandas as pd
import re
import random
import string

# 작업 디렉토리 및 경로 설정
workspace_path = '/workspace'
os.chdir(workspace_path)
rfdiffusion_path = os.path.join(workspace_path, 'RFdiffusion')
if rfdiffusion_path not in sys.path:
    sys.path.append(rfdiffusion_path)

from colabdesign.shared.protein import pdb_to_string

# --- 2. 핵심 함수 정의 ---
def run_rfdiffusion(command):
    log_filename = "/workspace/rfdiffusion_log.txt"
    print("⏳ RFdiffusion 프로세스를 시작합니다. 시간이 오래 걸릴 수 있습니다.")
    print(f"   (자세한 로그는 '{log_filename}' 파일에 저장됩니다)")
    
    os.system(f'{command} > {log_filename} 2>&1')
    
    print("✅ RFdiffusion 프로세스가 완료되었습니다.")

# --- 3. ⭐️ 파라미터 설정 (이곳만 수정하세요) ---
name = "0903-binder-final"
rf_contigs = "C311-391/0 100-100"
hotspot = "C331,C360,C378,C381,C323"
pdb_filename = "4k9e_c.pdb"

# RFdiffusion 설정
rf_iterations = 150
rf_num_designs = 20 # 필요시 늘리세요.

# ProteinMPNN & AlphaFold2 설정
mpnn_num_seqs = 8
mpnn_num_recycles = 3
mpnn_initial_guess = True
mpnn_use_multimer = True
mpnn_rm_aa = "C"
mpnn_sampling_temp = 0.1

# --- 4. 실행 경로 설정 및 RFdiffusion 명령어 생성 ---
pdb_path = os.path.join(workspace_path, pdb_filename)
base_name = name
output_dir = f"outputs/{base_name}"

while os.path.exists(output_dir):
    random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))
    base_name = f"{name}_{random_suffix}"
    output_dir = f"outputs/{base_name}"
os.makedirs(output_dir, exist_ok=True)
print(f"📂 모든 결과는 '{output_dir}' 폴더에 저장됩니다.")

opts = [
    f"inference.output_prefix={output_dir}/design",
    f"inference.num_designs={rf_num_designs}",
    f"diffuser.T={rf_iterations}",
    f"'contigmap.contigs=[{rf_contigs}]'",
    f"+inference.schedule_directory_path={output_dir}/schedules",
    f"inference.input_pdb={pdb_path}",
    f"ppi.hotspot_res=[{hotspot}]"
]
run_script_path = "/workspace/RFdiffusion/run_inference.py"
rfdiffusion_cmd = f"python {run_script_path} {' '.join(opts)}"

# --- 5. RFdiffusion 실행 ---
run_rfdiffusion(rfdiffusion_cmd)

# --- 6. ProteinMPNN용 Contig 자동 변환 ---
print("\n🔄 ProteinMPNN에 사용할 Contig 정보를 자동 변환합니다...")
mpnn_contigs_list = []
chain_code = ord('A')

for segment in rf_contigs.split('/0'):
    sub_parts = segment.strip().split()
    if not sub_parts: continue

    chain_sub_parts = []
    for sub_part in sub_parts:
        chain = chr(chain_code)
        length = 0
        if '-' in sub_part and sub_part[0].isalpha():
            match = re.match(r'[A-Z](\d+)-(\d+)', sub_part)
            start, end = int(match.group(1)), int(match.group(2))
            length = end - start + 1
        elif '-' in sub_part and sub_part[0].isdigit():
            length = int(sub_part.split('-')[0])
        
        if length > 0:
            chain_sub_parts.append(f'{chain}1-{length}')

    if chain_sub_parts:
        mpnn_contigs_list.append(" ".join(chain_sub_parts))
    
    chain_code += 1

mpnn_contigs = "/0".join(mpnn_contigs_list)
# ❗️ 여기가 최종 수정된 부분입니다.
mpnn_contigs_str = mpnn_contigs.replace('/0', ':')

print(f"   - 원본 Contig: {rf_contigs}")
print(f"   - ✨ 변환된 Contig: {mpnn_contigs}  (MPNN 명령어용: {mpnn_contigs_str})")

# --- 7. ProteinMPNN 및 AlphaFold2 실행 ---
print("\n" + "="*50)
print(" 🔬 ProteinMPNN 및 AlphaFold2 검증을 시작합니다. ")
print("="*50 + "\n")

opts_mpnn = [
    f"--pdb={output_dir}/design_0.pdb",
    f"--loc={output_dir}",
    f"--contig={mpnn_contigs_str}", # ✨ 최종 수정된 Contig 사용
    f"--num_designs={rf_num_designs}",
    f"--num_seqs={mpnn_num_seqs}",
    f"--num_recycles={mpnn_num_recycles}",
    f"--rm_aa={mpnn_rm_aa}",
    f"--mpnn_sampling_temp={mpnn_sampling_temp}"
]
if mpnn_initial_guess: opts_mpnn.append("--initial_guess")
if mpnn_use_multimer: opts_mpnn.append("--use_multimer")
mpnn_cmd = f"python -m colabdesign.rf.designability_test {' '.join(opts_mpnn)}"

os.system(mpnn_cmd)

# --- 8. 최종 결과 종합 및 출력 ---
print("\n" + "="*50)
print(" 📊 모든 작업 완료. 최종 결과를 요약합니다. ")
print("="*50 + "\n")

all_results_path = f"{output_dir}/all_results.csv"
if not os.path.exists(all_results_path):
    print("⚠️ 결과 파일(all_results.csv)이 생성되지 않았습니다. 로그를 확인해주세요.")
else:
    df = pd.read_csv(all_results_path)
    df_sorted = df.sort_values(by=['plddt', 'rmsd'], ascending=[False, True]).reset_index(drop=True)
    
    print("\n📊 상위 10개 디자인 결과:")
    print(df_sorted.head(10)[['design', 'n', 'plddt', 'rmsd']].to_string(index=False))

    if not df_sorted.empty:
      best_model = df_sorted.iloc[0]
      best_design_num = int(best_model['design'])
      best_seq_num = int(best_model['n'])
      
      print("\n" + "-"*50)
      print(f"🏆 Best Model: Design {best_design_num}, Sequence {best_seq_num}")
      print(f"   - pLDDT: {best_model['plddt']:.2f}, RMSD: {best_model['rmsd']:.2f}")
      print(f"   - 최종 PDB 파일 위치: {output_dir}/design_{best_design_num}_n{best_seq_num}.pdb")
      print("-"*50)

📂 모든 결과는 'outputs/0903-binder-design_fhoq' 폴더에 저장됩니다.
⏳ RFdiffusion 프로세스를 시작합니다. 시간이 오래 걸릴 수 있습니다.
   (자세한 로그는 '/workspace/rfdiffusion_log.txt' 파일에 저장됩니다)
✅ RFdiffusion 프로세스가 완료되었습니다.

🔄 ProteinMPNN에 사용할 Contig 정보를 자동 변환합니다...
   - 원본 Contig: C311-391/0 100-100
   - ✨ 변환된 Contig: A1-81/0B1-100  (MPNN 명령어용: A1-81:/0:B1-100)

 🔬 ProteinMPNN 및 AlphaFold2 검증을 시작합니다. 



Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/workspace/ColabDesign/colabdesign/rf/designability_test.py", line 199, in <module>
    main(sys.argv[1:])
  File "/workspace/ColabDesign/colabdesign/rf/designability_test.py", line 70, in main
    info = [get_info(x) for x in contigs]
  File "/workspace/ColabDesign/colabdesign/rf/designability_test.py", line 70, in <listcomp>
    info = [get_info(x) for x in contigs]
  File "/workspace/ColabDesign/colabdesign/rf/designability_test.py", line 18, in get_info
    for n,(a,b) in enumerate(sub_contigs):
ValueError: not enough values to unpack (expected 2, got 1)


{'pdb':'outputs/0903-binder-design_fhoq/design_0.pdb','loc':'outputs/0903-binder-design_fhoq','contigs':'A1-81:/0:B1-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recycles':3,'rm_aa':'C','num_designs':2,'mpnn_sampling_temp':0.1}

 📊 모든 작업 완료. 최종 결과를 요약합니다. 

⚠️ 결과 파일(all_results.csv)이 생성되지 않았습니다. 로그를 확인해주세요.
